In [22]:
import pandas as pd
import numpy as np
import util

enrollment_path = './data/tx/enrollment/raw/'

enrollment_files = [
#     'Enrollment Report_Statewide_Campuses_Grade_2012-2013.csv'
#     , 'Enrollment Report_Statewide_Campuses_Grade_2013-2014.csv'
    'Enrollment Report_Statewide_Campuses_Grade_2014-2015.csv'
    , 'Enrollment Report_Statewide_Campuses_Grade_2015-2016.csv'
    , 'Enrollment Report_Statewide_Campuses_Grade_2016-2017.csv'
    , 'Enrollment Report_Statewide_Campuses_Grade_2017-2018.csv'
    , 'Enrollment Report_Statewide_Campuses_Grade_2018-2019.csv'
]

ethnicity_files = [
#     'Enrollment Report_Statewide_Campuses_Grade_Ethnicity_2012-2013.csv'
#     , 'Enrollment Report_Statewide_Campuses_Grade_Ethnicity_2013-2014.csv'
    'Enrollment Report_Statewide_Campuses_Grade_Ethnicity_2014-2015.csv'
    , 'Enrollment Report_Statewide_Campuses_Grade_Ethnicity_2015-2016.csv'
    , 'Enrollment Report_Statewide_Campuses_Grade_Ethnicity_2016-2017.csv'
    , 'Enrollment Report_Statewide_Campuses_Grade_Ethnicity_2017-2018.csv'
    , 'Enrollment Report_Statewide_Campuses_Grade_Ethnicity_2018-2019.csv'
]

li_files = [
    'StudPgmStateCampus15state.csv'
    , 'StudPgmStateCampus16state.csv'
    , 'StudPgmStateCampus17state.csv'
    , 'StudPgmStateCampus18state.csv'
    , 'StudPgmStateCampus19state.csv'
]

years = [x[48:52] for x in enrollment_files]
years_li = ['2015', '2016', '2017', '2018', '2019']

df = pd.read_csv(enrollment_path + enrollment_files[0], skiprows = 4)
df['year'] = df.apply(lambda x: years[0], axis=1)

for i in range(1, len(enrollment_files)):
    df2 = pd.read_csv(enrollment_path + enrollment_files[i], skiprows = 4)
    df2['year'] = df2.apply(lambda x: years[i], axis=1)
    df = df.append(df2, ignore_index = True, sort = True)
    
df_eth = pd.read_csv(enrollment_path + ethnicity_files[0], skiprows = 4)
df_eth['year'] = df_eth.apply(lambda x: years[0], axis=1)

for i in range(1, len(ethnicity_files)):
    df2 = pd.read_csv(enrollment_path + ethnicity_files[i], skiprows = 4)
    df2['year'] = df2.apply(lambda x: years[i], axis=1)
    df_eth = df_eth.append(df2, ignore_index = True, sort = True)
    
df_li = pd.DataFrame()

for i in range(0, len(li_files)):
    df2 = pd.read_csv(enrollment_path + li_files[i], skiprows = 6)
    df2['year'] = df2.apply(lambda x: years_li[i], axis=1)
    df_li = df_li.append(df2, ignore_index = True, sort = True)

In [23]:
# rename fields in normal and ethnicity and li files
df = df.rename(columns = {
    'District Number': 'district_id'
    , 'District Name': 'district'
    , 'Campus Number': 'school_id'
    , 'Campus Name': 'school'
    , 'Grade Level Name': 'grade'
    , 'Enrollment by Grade Level': 'num'
})

df_eth = df_eth.rename(columns = {
    'District Number': 'district_id'
    , 'District Name': 'district'
    , 'Campus Number': 'school_id'
    , 'Campus Name': 'school'
    , 'Ethnicity Name': 'group_state'
    , 'Grade Level Name': 'grade'
    , 'Ethnicity Count': 'num'
})

df_all = df_li[['year', 'DISTRICT NUMBER', 'DISTRICT NAME', 'CAMPUS NUMBER', 'CAMPUS NAME', 'TOTAL ENROLLMENT']]
df_all = df_all.rename(columns = {
    'DISTRICT NUMBER': 'district_id'
    , 'DISTRICT NAME': 'district'
    , 'CAMPUS NUMBER': 'school_id'
    , 'CAMPUS NAME': 'school'
    , 'TOTAL ENROLLMENT': 'num'
})

df_li = df_li.rename(columns = {
    'DISTRICT NUMBER': 'district_id'
    , 'DISTRICT NAME': 'district'
    , 'CAMPUS NUMBER': 'school_id'
    , 'CAMPUS NAME': 'school'
    , 'TOTAL ECONOMICALLY DISADVANTAGED STUDENTS': 'num'
})

In [24]:
# add All Groups field to main data and 'All Grades' field to df_li
df['group_state'] = df.apply(lambda x: 'All Groups', axis = 1)
df_li['group_state'] = 'Low-Income'
df_li['grade'] = 'All Grades'
df_all['grade'] = 'All Grades'
df_all['group_state'] = 'All Groups'

In [25]:
# create 'All Grades' rollup for ethnicity
df_eth_all = df_eth
df_eth_all = df_eth_all.groupby(['year', 'district_id', 'district', 'school_id', 'school', 'group_state'], as_index=False)['num'].sum()
df_eth_all['grade'] = 'All Grades'

In [26]:
# append data
df = df.append(df_eth, ignore_index = True, sort = True)
df = df.append(df_eth_all, ignore_index = True, sort = True)
df = df.append(df_li, ignore_index = True, sort = True)
df = df.append(df_all, ignore_index = True, sort = True)

In [27]:
# drop unneeded columns
df = df[['year', 'district_id', 'district', 'school_id', 'school', 'grade', 'group_state', 'num']]

In [28]:
# drop rows with a null school_id
print(df.shape)
df = df[~df['school_id'].isna()]
print(df.shape)

(1512474, 8)
(1512438, 8)


In [29]:
# clean up grades
df['grade'] = df['grade'].str.replace('  ', ' ')

In [30]:
def toInt(row, column):
    try:
        return int(row[column])
    except:
        return np.nan
df['num'] = df.apply(lambda x: toInt(x, 'num'), axis = 1)

In [31]:
# clean 0s and nulls
print(df.shape)
df = df.dropna(subset=['num'])
df = df[df['num'] > 0]
print(df.shape)

(1512438, 8)
(1013468, 8)


In [32]:
print(df.dtypes)
df['district_id'] = df['district_id'].astype(int)
df['district_id'] = df['district_id'].astype(str)
df['school_id'] = df['school_id'].astype(int)
df['school_id'] = df['school_id'].astype(str)
df['num'] = df['num'].astype('int64')
print(df.dtypes)

year            object
district_id    float64
district        object
school_id      float64
school          object
grade           object
group_state     object
num            float64
dtype: object
year           object
district_id    object
district       object
school_id      object
school         object
grade          object
group_state    object
num             int64
dtype: object


In [35]:
# export results
df.to_csv('./data/finalized/tx_enrollment.csv', index=False)